In [ ]:
import pandas as pd
import pickle
import numpy as np
from brainflow.board_shim import BoardShim, BrainFlowInputParams, LogLevels, BoardIds
from brainflow.data_filter import DataFilter, FilterTypes
import matplotlib
import os
import torch
import pytorch_lightning as pl
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
from pytorch_lightning.loggers import WandbLogger

import cv2
from torch.utils.data import DataLoader
import wandb

matplotlib.use('Agg')
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
%matplotlib inline
scaler = StandardScaler()

In [ ]:
wandb.login()
api_key = 'a45abb01f9556b57620ce77c8984452bee7a8772'
board_id = 38
eeg_names = ['TP9', 'Fp1', 'Fp2', 'TP10', 'AUX']
sf = 256

In [ ]:
class PickleLatentDatasetLoader(Dataset):
    def __init__(self, annotations_file, dir):
        self.file_lists = pd.read_csv(annotations_file, header=None)
        self.dir = dir

    def __len__(self):
        return len(self.file_lists)

    def __getitem__(self, idx):
        file_path = os.path.join(self.dir, self.file_lists.iloc[idx, 0].replace('/','\\'))
        label = self.file_lists.iloc[idx, 1]

        pkl_file = open(file_path, 'rb')
        results = pickle.load(pkl_file)
        results = np.array(results)
        pkl_file.close()

        results = torch.tensor(results)
        label = torch.tensor(label).type(torch.FloatTensor)

        return (results, label)

In [ ]:
pkl_dir = 'D:\\Nicko\\TUGAS_AKHIR\\Dataset\\Dataset_TA_pkl\\LATENT_32640_SINGLE\\'
path_file = pkl_dir + 'path_and_label.csv'

In [ ]:
datasets = PickleLatentDatasetLoader(
    annotations_file=path_file,
    dir=pkl_dir
)

In [ ]:
latent_loaded = datasets[0]

In [ ]:
latent_loaded[0].shape

In [ ]:
# split train to train and validation
# use 20% of training data for validation
train_set_size = int(len(datasets) * 0.7)
valid_set_size = int(len(datasets) * 0.2)
test_set_size = len(datasets) - train_set_size - valid_set_size

# split the train set into two
seed = torch.Generator().manual_seed(42)
train_set, valid_set, test_set = torch.utils.data.random_split(datasets, [train_set_size, valid_set_size, test_set_size], generator=seed)

# data loader
train_dataloader = DataLoader(train_set, batch_size=10, shuffle=True)
validation_dataloader = DataLoader(valid_set, batch_size=10)
test_dataloader = DataLoader(test_set, batch_size=10)

In [ ]:
#MODEL 1 (Flatten and FC only)
class LitClassifier(pl.LightningModule):
    def __init__(self, input_shape, learning_rate=1e-4):
        super().__init__()

        # log hyperparameters
        self.save_hyperparameters()
        self.learning_rate = learning_rate
        self.loss_fn = torch.nn.BCEWithLogitsLoss()

        # Convolutional
        self.conv_layer = torch.nn.Sequential(
            # torch.nn.Conv2d(64, 128, 3, stride=1, padding=0),
            # torch.nn.ReLU(True),
            # torch.nn.Conv2d(32, 64, 3, stride=1, padding=0),
            # torch.nn.ReLU(True),
            # torch.nn.Conv2d(64, 64, 3, stride=1, padding=0),
            # torch.nn.ReLU(True)
        )

        # Flatten
        self.flatten_layer = torch.nn.Flatten(start_dim=1)

        # Fully Connected
        n_sizes = self._get_output_shape(input_shape)
        self.fc_layer = torch.nn.Sequential(
            torch.nn.Linear(n_sizes, 5120),
            torch.nn.ReLU(True),
            torch.nn.Linear(5120, 1280),
            torch.nn.ReLU(True),
            torch.nn.Linear(1280, 512),
            torch.nn.Dropout(0.2),
            torch.nn.Linear(512, 1)
        )

    def _get_output_shape(self, shape):
        batch_size = 50
        input = torch.autograd.Variable(torch.rand(batch_size, *shape))
        output_feat = self.conv_layer(input)
        n_size = output_feat.data.view(batch_size, -1).size(1)
        return n_size

    def forward(self, x):
         x = self.conv_layer(x)
         x_flat = self.flatten_layer(x)
         y_logits = self.fc_layer(x_flat)
         y_pred = torch.round(torch.sigmoid(y_logits))
         return y_pred

    def training_step(self, batch, batch_idx):
        x, y = batch
        x = self.conv_layer(x)
        x_flat = self.flatten_layer(x)
        y_logits = self.fc_layer(x_flat)
        y_logits = y_logits.squeeze()
        loss = self.loss_fn(y_logits, y)
        self.log("train_loss", loss, on_epoch=True)
        return loss

    def validation_step(self, val_batch, batch_idx):
        x, y = val_batch
        x = self.conv_layer(x)
        x_flat = self.flatten_layer(x)
        y_logits = self.fc_layer(x_flat)
        y_logits = y_logits.squeeze()
        val_loss = self.loss_fn(y_logits, y)
        self.log("val_loss", val_loss, on_epoch=True)

    def test_step(self, test_batch, batch_idx):
        x, y = test_batch
        x = self.conv_layer(x)
        x_flat = self.flatten_layer(x)
        y_logits = self.fc_layer(x_flat)
        y_logits = y_logits.squeeze()
        test_loss = self.loss_fn(y_logits, y)
        self.log("test_loss", test_loss)

    def configure_optimizers(self):
        optimizer = torch.optim.SGD(self.parameters(), lr=self.learning_rate)
        return optimizer

In [ ]:
classifier = LitClassifier(input_shape=(64,10,51))

In [ ]:
checkpoint_callback = pl.callbacks.ModelCheckpoint(monitor="val_loss", dirpath="D:\\Nicko\\TUGAS_AKHIR\\Classifier\\model_3\\classifier\\run_1",
    filename="classifier-{epoch:02d}-{val_loss:.2f}",)

In [ ]:
wandb_logger = WandbLogger(project='classifier', save_dir='D:\\Nicko\\TUGAS_AKHIR\\Classifier\\model_3')

In [ ]:
trainer = pl.Trainer(max_epochs=1000, devices=1, accelerator='gpu', log_every_n_steps=9, logger=wandb_logger, callbacks=[checkpoint_callback])
trainer.fit(classifier, train_dataloader, validation_dataloader)
# wandb.finish()

In [ ]:
trainer.test(classifier, test_dataloader)